In [1]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, clone_model

In [2]:
ALPHA = 1  # ou o valor usado

In [3]:
with open(f'dataset/federated_datasets_dirichlet_{ALPHA}.pkl', 'rb') as f:
    data = pickle.load(f)

test_dataset = data['data_global_test']
X_test=test_dataset['X']
y_test=test_dataset['y']

# Garantir (N, T, 1)
if X_test.ndim == 2:
    X_test = np.expand_dims(X_test, axis=-1)

print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_test: (7200, 1536, 1)
y_test: (7200,)


In [4]:
model_seq = load_model(f'resultados_federado/global_model_{ALPHA}.h5')
model_seq.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1536, 32)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization             │ (None, 1536, 32)       │            64 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 768, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 768, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_1           │ (None, 768, 64)        │           128 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 384, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 384, 128)       │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_2           │ (None, 384, 128)       │           256 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 192, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,650 (303.32 KB)

 Trainable params: 77,648 (303.31 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [5]:
input_shape = X_test.shape[1:]  # (T, 1)

# Clonar arquitetura
model_func = clone_model(
    model_seq,
    input_tensors=tf.keras.Input(shape=input_shape, name="gradcam_input")
)

# Copiar pesos
model_func.set_weights(model_seq.get_weights())

model_func.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1536, 32)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization             │ (None, 1536, 32)       │            64 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 768, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 768, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_1           │ (None, 768, 64)        │           128 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 384, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 384, 128)       │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_2           │ (None, 384, 128)       │           256 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 192, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 232,946 (909.95 KB)

 Trainable params: 77,648 (303.31 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 155,298 (606.64 KB)

In [6]:
def grad_cam_1d(model, signal, class_index=None, conv_layer_name=None):
    """
    Grad-CAM 1D robusto para séries temporais
    """

    # Encontrar última Conv1D
    if conv_layer_name is None:
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv1D):
                conv_layer_name = layer.name
                break

    if conv_layer_name is None:
        raise ValueError("Nenhuma camada Conv1D encontrada.")

    conv_layer = model.get_layer(conv_layer_name)

    # Modelo intermediário
    conv_model = tf.keras.Model(
        inputs=model.input,
        outputs=conv_layer.output
    )

    with tf.GradientTape() as tape:
        conv_outputs = conv_model(signal)
        tape.watch(conv_outputs)

        predictions = model(signal, training=False)

        if class_index is None:
            class_index = tf.argmax(predictions[0])

        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)

    # GAP no tempo
    weights = tf.reduce_mean(grads, axis=1)

    cam = tf.reduce_sum(weights[:, :, tf.newaxis] * conv_outputs, axis=-1)
    cam = tf.nn.relu(cam)

    cam = cam.numpy()[0]
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    return cam, int(class_index), conv_layer_name


In [ ]:
idx = 0

signal = X_test[idx:idx+1]
true_label = y_test[idx]

cam, pred_class, layer_used = grad_cam_1d(model_func, signal)

print(f"Classe real: {true_label}")
print(f"Classe predita: {pred_class}")
print(f"Camada usada: {layer_used}")
print("Shape CAM:", cam.shape)


In [7]:
idx = 0

signal = X_test[idx:idx+1]
true_label = y_test[idx]

cam, pred_class, layer_used = grad_cam_1d(model_func, signal)

print(f"Classe real: {true_label}")
print(f"Classe predita: {pred_class}")
print(f"Camada usada: {layer_used}")
print("Shape CAM:", cam.shape)


AttributeError: The layer sequential has never been called and thus has no defined input.